# 11.8번 문제 풀이

In [245]:
import tensorflow as tf
import numpy as np

In [246]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs_HW11"
logdir = "{}/run-{}/".format(root_logdir, now)

In [247]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist=input_data.read_data_sets("MNIST")

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [248]:
reduced_train = mnist.train.labels < 5
reduced_valid = mnist.validation.labels < 5
#print (reduced_valid)
mnist_train_images_reduced = mnist.train.images[reduced_train]
mnist_train_labels_reduced = mnist.train.labels[reduced_train]
mnist_valid_images_reduced = mnist.validation.images[reduced_valid]
mnist_valid_labels_reduced = mnist.validation.labels[reduced_valid]
mnist_train_labels_reduced.shape
print (mnist_train_labels_reduced[0])

3


In [249]:
n_inputs = 28*28
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_hidden4 = 100
n_hidden5 = 100
n_outputs = 5

In [250]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name = "X") # 이미 scaled 되어있는 데이터가 들어갈 예정
Y = tf.placeholder(tf.int32, shape=(None), name="Y")

In [251]:
# DNN 만들어주는 파트, name_scope로 만들어주는 부분들의 이름들을 하나로 묶음

he_init = tf.contrib.layers.variance_scaling_initializer()
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, name="hidden3", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden4 = tf.layers.dense(hidden3, n_hidden4, name="hidden4", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, name="hidden5", activation=tf.nn.elu, kernel_initializer=he_init)
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs", kernel_initializer=he_init)
    logits_out = tf.nn.softmax(logits)

In [252]:
# DNN에서 평가를 위한 loss파트 정의
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
    cost = tf.reduce_mean(xentropy, name="cost")

In [253]:
# DNN 훈련파트
learning_rate = 0.01
with tf.name_scope("train"):
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(cost)

In [254]:
## DNN 평가를 위한 파트
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, Y, 1) ## 
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) ## type을 bool에서 float32로 바꿔줌

In [255]:
n_epochs = 30
batch_size = 1000 ## 최대한 트레이닝 샘플 개수의 약수로 나오게 현재는 28038개

In [256]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

for op in (X,Y,accuracy,training_op):
    tf.add_to_collection("my_important_ops", op)

cost_summary = tf.summary.scalar('cost', cost)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_batches = mnist_train_images_reduced.shape[0] // batch_size

In [257]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        #for iteration in range(mnist.train.num_examples // batch_size):
        #    X_batch, Y_batch = mnist.train.next_batch(batch_size) ## 기본적으로 제공하는 파트
        shuffled_idx = np.random.permutation(mnist_train_images_reduced.shape[0])
        X_batches = np.array_split(mnist_train_images_reduced[shuffled_idx], n_batches)
        Y_batches = np.array_split(mnist_train_labels_reduced[shuffled_idx], n_batches)
        batch_index = 0
        for X_batch, Y_batch in zip(X_batches, Y_batches):
            sess.run(training_op, feed_dict={X:X_batch, Y:Y_batch})
            #print ((len(Y_batch[0])))
            batch_index += 1
            #print (cost)
            loss = cost.eval(feed_dict={X:X_batch, Y:Y_batch}) # iteration 돌릴때 cost=cost.eval(~)f로 하면 중복되어버림 즉 cost가 numpy float으로 선언되어버림
            #acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, Y:mnist.validation.labels})
            step = epoch * n_batches + batch_index
            summary_str = cost_summary.eval(feed_dict={X: X_batch, Y: Y_batch})
            file_writer.add_summary(summary_str, step)
        acc_train = accuracy.eval(feed_dict={X:X_batch, Y:Y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist_valid_images_reduced, Y:mnist_valid_labels_reduced})
        #print (epoch, "Train accuracy:", acc_train, "Cost:", loss)
        print(epoch, "Train accuracy:", acc_train, "Validation accuracy:", acc_val)
    save_path = saver.save(sess, "./Ch11_HW_final.ckpt")
    file_writer.close()

0 Train accuracy: 0.955045 Validation accuracy: 0.955043
1 Train accuracy: 0.96803194 Validation accuracy: 0.97341675
2 Train accuracy: 0.97202796 Validation accuracy: 0.97537136
3 Train accuracy: 0.985015 Validation accuracy: 0.9824081
4 Train accuracy: 0.987013 Validation accuracy: 0.9824081
5 Train accuracy: 0.986014 Validation accuracy: 0.9847537
6 Train accuracy: 0.988012 Validation accuracy: 0.9851446
7 Train accuracy: 0.991009 Validation accuracy: 0.98397183
8 Train accuracy: 0.995005 Validation accuracy: 0.9851446
9 Train accuracy: 0.996004 Validation accuracy: 0.98670834
10 Train accuracy: 0.991009 Validation accuracy: 0.98553556
11 Train accuracy: 0.994006 Validation accuracy: 0.9851446
12 Train accuracy: 0.998002 Validation accuracy: 0.9870993
13 Train accuracy: 0.995005 Validation accuracy: 0.9847537
14 Train accuracy: 0.995005 Validation accuracy: 0.9906177
15 Train accuracy: 0.997003 Validation accuracy: 0.9878812
16 Train accuracy: 0.999001 Validation accuracy: 0.9917905

## 배치정규화 사용

In [445]:
import tensorflow as tf
import numpy as np

In [446]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs_HW11"
logdir = "{}/run-{}/".format(root_logdir, now)

In [447]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist=input_data.read_data_sets("MNIST")

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [448]:
reduced_train = mnist.train.labels < 5
reduced_valid = mnist.validation.labels < 5
#print (reduced_valid)
mnist_train_images_reduced = mnist.train.images[reduced_train]
mnist_train_labels_reduced = mnist.train.labels[reduced_train]
mnist_valid_images_reduced = mnist.validation.images[reduced_valid]
mnist_valid_labels_reduced = mnist.validation.labels[reduced_valid]
mnist_train_labels_reduced.shape
print (mnist_train_labels_reduced[0])

3


In [449]:
n_inputs = 28*28
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_hidden4 = 100
n_hidden5 = 100
n_outputs = 5

In [450]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name = "X") # 이미 scaled 되어있는 데이터가 들어갈 예정
Y = tf.placeholder(tf.int32, shape=(None), name="Y")
training = tf.placeholder_with_default(False, shape=(), name="training")

In [451]:
# DNN 만들어주는 파트, name_scope로 만들어주는 부분들의 이름들을 하나로 묶음

he_init = tf.contrib.layers.variance_scaling_initializer()
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")#, activation=tf.nn.elu)#, kernel_initializer=he_init)
    bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
    bn1_act = tf.nn.elu(bn1)
    hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")#, activation=tf.nn.elu)#, kernel_initializer=he_init)
    bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
    bn2_act = tf.nn.elu(bn2)
    hidden3 = tf.layers.dense(bn2_act, n_hidden3, name="hidden3")#, activation=tf.nn.elu)#, kernel_initializer=he_init)
    bn3 = tf.layers.batch_normalization(hidden3, training=training, momentum=0.9)
    bn3_act = tf.nn.elu(bn3)
    hidden4 = tf.layers.dense(bn3_act, n_hidden4, name="hidden4")#, activation=tf.nn.elu)#, kernel_initializer=he_init)
    bn4 = tf.layers.batch_normalization(hidden4, training=training, momentum=0.9)
    bn4_act = tf.nn.elu(bn4)
    hidden5 = tf.layers.dense(bn4_act, n_hidden5, name="hidden5")#, activation=tf.nn.elu)#, kernel_initializer=he_init)
    bn5 = tf.layers.batch_normalization(hidden5, training=training, momentum=0.9)
    bn5_act = tf.nn.elu(bn5)
    logits_before_bn = tf.layers.dense(bn5_act, n_outputs, name="outputs")#, kernel_initializer=he_init)
    logits = tf.layers.batch_normalization(logits_before_bn, training=training, momentum=0.9)
    logits_out = tf.nn.softmax(logits)

In [452]:
# DNN에서 평가를 위한 loss파트 정의
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
    cost = tf.reduce_mean(xentropy, name="cost")

In [453]:
# DNN 훈련파트
learning_rate = 0.01
with tf.name_scope("train"):
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(cost)

In [454]:
## DNN 평가를 위한 파트
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, Y, 1) ## 
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) ## type을 bool에서 float32로 바꿔줌

In [455]:
n_epochs = 30
batch_size = 1000 ## 최대한 트레이닝 샘플 개수의 약수로 나오게 현재는 28038개

In [71]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

for op in (X,Y,accuracy,training_op):
    tf.add_to_collection("my_important_ops", op)
    

cost_summary = tf.summary.scalar('cost', cost)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_batches = mnist_train_images_reduced.shape[0] // batch_size

ValueError: At least two variables have the same name: hidden1/bias/Adam

In [457]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(mnist_train_images_reduced.shape[0])
        X_batches = np.array_split(mnist_train_images_reduced[shuffled_idx], n_batches)
        Y_batches = np.array_split(mnist_train_labels_reduced[shuffled_idx], n_batches)
        batch_index = 0
        for X_batch, Y_batch in zip(X_batches, Y_batches):
            sess.run([training_op, extra_update_ops], feed_dict={X:X_batch, Y:Y_batch})
            #print ((len(Y_batch[0])))
            batch_index += 1
            #print (cost)
            loss = cost.eval(feed_dict={X:X_batch, Y:Y_batch}) # iteration 돌릴때 cost=cost.eval(~)f로 하면 중복되어버림 즉 cost가 numpy float으로 선언되어버림
            #acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, Y:mnist.validation.labels})
            step = epoch * n_batches + batch_index
            summary_str = cost_summary.eval(feed_dict={training: True, X: X_batch, Y: Y_batch})
            file_writer.add_summary(summary_str, step)
        acc_train = accuracy.eval(feed_dict={X:X_batch, Y:Y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist_valid_images_reduced, Y:mnist_valid_labels_reduced})
        #print (epoch, "Train accuracy:", acc_train, "Cost:", loss)
        print(epoch, "Train accuracy:", acc_train, "Validation accuracy:", acc_val)
    save_path = saver.save(sess, "./Ch10_DNN_test_final.ckpt")
    file_writer.close()

0 Train accuracy: 0.95804197 Validation accuracy: 0.96129787
1 Train accuracy: 0.985015 Validation accuracy: 0.9741986
2 Train accuracy: 0.976024 Validation accuracy: 0.97263485
3 Train accuracy: 0.971029 Validation accuracy: 0.98279905
4 Train accuracy: 0.984016 Validation accuracy: 0.98631746
5 Train accuracy: 0.992008 Validation accuracy: 0.988663
6 Train accuracy: 0.993007 Validation accuracy: 0.988663
7 Train accuracy: 0.995005 Validation accuracy: 0.9878812
8 Train accuracy: 0.993007 Validation accuracy: 0.988663
9 Train accuracy: 0.994006 Validation accuracy: 0.9878812
10 Train accuracy: 0.997003 Validation accuracy: 0.98944485
11 Train accuracy: 0.997003 Validation accuracy: 0.98905396
12 Train accuracy: 0.996004 Validation accuracy: 0.99022675
13 Train accuracy: 0.997003 Validation accuracy: 0.9906177
14 Train accuracy: 1.0 Validation accuracy: 0.9913995
15 Train accuracy: 0.997003 Validation accuracy: 0.98905396
16 Train accuracy: 1.0 Validation accuracy: 0.9913995
17 Train a

 ## 드롭아웃 규제 사용해보기

In [4]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs_HW11"
logdir = "{}/run-{}/".format(root_logdir, now)

In [5]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist=input_data.read_data_sets("MNIST")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
reduced_train = mnist.train.labels < 5
reduced_valid = mnist.validation.labels < 5
#print (reduced_valid)
mnist_train_images_reduced = mnist.train.images[reduced_train]
mnist_train_labels_reduced = mnist.train.labels[reduced_train]
mnist_valid_images_reduced = mnist.validation.images[reduced_valid]
mnist_valid_labels_reduced = mnist.validation.labels[reduced_valid]
mnist_train_labels_reduced.shape
print (mnist_train_labels_reduced[0])

3


In [7]:
n_inputs = 28*28
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_hidden4 = 100
n_hidden5 = 100
n_outputs = 5

In [9]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name = "X") # 이미 scaled 되어있는 데이터가 들어갈 예정
Y = tf.placeholder(tf.int32, shape=(None), name="Y")
training = tf.placeholder_with_default(False, shape=(), name="training")

In [10]:
# DNN 만들어주는 파트, name_scope로 만들어주는 부분들의 이름들을 하나로 묶음

he_init = tf.contrib.layers.variance_scaling_initializer()

dropout_rate = 0.5
X_drop = tf.layers.dropout(X, dropout_rate, training=training)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X_drop, n_hidden1, name="hidden1", activation=tf.nn.elu)#, kernel_initializer=he_init)
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, name="hidden2", activation=tf.nn.elu)#, kernel_initializer=he_init)
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
    hidden3 = tf.layers.dense(hidden2_drop, n_hidden3, name="hidden3", activation=tf.nn.elu)#, kernel_initializer=he_init)
    hidden3_drop = tf.layers.dropout(hidden3, dropout_rate, training=training)
    hidden4 = tf.layers.dense(hidden3_drop, n_hidden4, name="hidden4", activation=tf.nn.elu)#, kernel_initializer=he_init)
    hidden4_drop = tf.layers.dropout(hidden4, dropout_rate, training=training)
    hidden5 = tf.layers.dense(hidden4_drop, n_hidden5, name="hidden5", activation=tf.nn.elu)#, kernel_initializer=he_init)
    hidden5_drop = tf.layers.dropout(hidden5, dropout_rate, training=training)
    logits = tf.layers.dense(hidden5_drop, n_outputs, name="outputs")
    logits_out = tf.nn.softmax(logits)

In [11]:
# DNN에서 평가를 위한 loss파트 정의
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
    cost = tf.reduce_mean(xentropy, name="cost")

In [12]:
# DNN 훈련파트
learning_rate = 0.01
with tf.name_scope("train"):
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(cost)

In [13]:
## DNN 평가를 위한 파트
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, Y, 1) ## 
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) ## type을 bool에서 float32로 바꿔줌

In [14]:
n_epochs = 30
batch_size = 1000 ## 최대한 트레이닝 샘플 개수의 약수로 나오게 현재는 28038개

In [15]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

cost_summary = tf.summary.scalar('cost', cost)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_batches = mnist_train_images_reduced.shape[0] // batch_size

In [16]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(mnist_train_images_reduced.shape[0])
        X_batches = np.array_split(mnist_train_images_reduced[shuffled_idx], n_batches)
        Y_batches = np.array_split(mnist_train_labels_reduced[shuffled_idx], n_batches)
        batch_index = 0
        for X_batch, Y_batch in zip(X_batches, Y_batches):
            sess.run(training_op, feed_dict={training: True, X:X_batch, Y:Y_batch})
            #print ((len(Y_batch[0])))
            batch_index += 1
            #print (cost)
            loss = cost.eval(feed_dict={X:X_batch, Y:Y_batch}) # iteration 돌릴때 cost=cost.eval(~)f로 하면 중복되어버림 즉 cost가 numpy float으로 선언되어버림
            #acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, Y:mnist.validation.labels})
            step = epoch * n_batches + batch_index
            summary_str = cost_summary.eval(feed_dict={ X: X_batch, Y: Y_batch})
            file_writer.add_summary(summary_str, step)
        acc_train = accuracy.eval(feed_dict={X:X_batch, Y:Y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist_valid_images_reduced, Y:mnist_valid_labels_reduced})
        #print (epoch, "Train accuracy:", acc_train, "Cost:", loss)
        print(epoch, "Train accuracy:", acc_train, "Validation accuracy:", acc_val)
    save_path = saver.save(sess, "./Ch11_HW_final.ckpt")
    file_writer.close()

0 Train accuracy: 0.96303695 Validation accuracy: 0.9589523
1 Train accuracy: 0.969031 Validation accuracy: 0.96598905
2 Train accuracy: 0.96803194 Validation accuracy: 0.97224396
3 Train accuracy: 0.971029 Validation accuracy: 0.9741986
4 Train accuracy: 0.97202796 Validation accuracy: 0.9741986
5 Train accuracy: 0.98002 Validation accuracy: 0.9757623
6 Train accuracy: 0.976024 Validation accuracy: 0.97810787
7 Train accuracy: 0.97902095 Validation accuracy: 0.98279905
8 Train accuracy: 0.977023 Validation accuracy: 0.9859265
9 Train accuracy: 0.989011 Validation accuracy: 0.98553556
10 Train accuracy: 0.98301697 Validation accuracy: 0.9847537
11 Train accuracy: 0.986014 Validation accuracy: 0.9824081
12 Train accuracy: 0.988012 Validation accuracy: 0.98319
13 Train accuracy: 0.988012 Validation accuracy: 0.98670834
14 Train accuracy: 0.988012 Validation accuracy: 0.9851446
15 Train accuracy: 0.985015 Validation accuracy: 0.98749024
16 Train accuracy: 0.99000996 Validation accuracy: 0

 # 11.9 번 문제 풀이

## 단순재사용

In [348]:
tf.reset_default_graph()
#saver_meta = tf.train.import_meta_graph("./Ch11_HW_final.ckpt.meta")

### 중요: 계속 불러들여올때 마다 중복되어버림 즉 reset을 해주지 않으면 계속하여 쌓여버림

In [349]:
for op in tf.get_default_graph().get_operations():
    print (op.name)
#for var in tf.get_default_graph().get_tensor_by_name():
#    print (var)

In [350]:
#X = tf.get_default_graph().get_tensor_by_name("X:0")
#Y = tf.get_default_graph().get_tensor_by_name("Y:0")
#accuracy = tf.get_default_graph().get_tensor_by_name("eval/Mean:0")
#training_op = tf.get_default_graph().get_operation_by_name("train/Adam")

In [351]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist=input_data.read_data_sets("MNIST")

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [352]:
reduced_train = mnist.train.labels < 5
reduced_valid = mnist.validation.labels < 5
#print (reduced_valid)
mnist_train_images_reduced = mnist.train.images[reduced_train]
mnist_train_labels_reduced = mnist.train.labels[reduced_train]
mnist_valid_images_reduced = mnist.validation.images[reduced_valid]
mnist_valid_labels_reduced = mnist.validation.labels[reduced_valid]

In [353]:
n_inputs = 28*28
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_hidden4 = 100
n_hidden5 = 100
n_outputs = 5

In [354]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name = "X") # 이미 scaled 되어있는 데이터가 들어갈 예정
Y = tf.placeholder(tf.int32, shape=(None), name="Y")

In [355]:
# DNN 만들어주는 파트, name_scope로 만들어주는 부분들의 이름들을 하나로 묶음

he_init = tf.contrib.layers.variance_scaling_initializer()
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, name="hidden3", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden4 = tf.layers.dense(hidden3, n_hidden4, name="hidden4", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, name="hidden5", activation=tf.nn.elu, kernel_initializer=he_init)
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs", kernel_initializer=he_init)
    logits_out = tf.nn.softmax(logits)

In [356]:
# DNN에서 평가를 위한 loss파트 정의
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
    cost = tf.reduce_mean(xentropy, name="cost")

In [357]:
# DNN 훈련파트
learning_rate = 0.01
with tf.name_scope("train"):
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(cost)

In [358]:
## DNN 평가를 위한 파트
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, Y, 1) ## 
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) ## type을 bool에서 float32로 바꿔줌

In [359]:
n_epochs = 30
batch_size = 1000 ## 최대한 트레이닝 샘플 개수의 약수로 나오게 현재는 28038개

In [363]:
### 재사용할 레이어 불러오기
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="hidden[12345]") # scope에 아무것도 안하면 전체 다 불러옴 즉 마지막 출력층 (ouput)까지 불러옴
print (reuse_vars) # 출력해보면 어떤게 나오는지 감이 잡힘
restore_saver = tf.train.Saver(reuse_vars)

[<tf.Variable 'hidden1/kernel:0' shape=(784, 100) dtype=float32_ref>, <tf.Variable 'hidden1/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden2/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'hidden2/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden3/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'hidden3/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden4/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'hidden4/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden5/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'hidden5/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden1/kernel/Adam:0' shape=(784, 100) dtype=float32_ref>, <tf.Variable 'hidden1/kernel/Adam_1:0' shape=(784, 100) dtype=float32_ref>, <tf.Variable 'hidden1/bias/Adam:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden1/bias/Adam_1:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden2/kernel/Adam:0' shape=(100, 10

In [365]:
reuse_vars2 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[12345]")
print (reuse_vars2) # 출력해보면 어떤게 나오는지 감이 잡힘
#restore_saver = tf.train.Saver(reuse_vars2)

[<tf.Variable 'hidden1/kernel:0' shape=(784, 100) dtype=float32_ref>, <tf.Variable 'hidden1/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden2/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'hidden2/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden3/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'hidden3/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden4/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'hidden4/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden5/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'hidden5/bias:0' shape=(100,) dtype=float32_ref>]


In [366]:
init = tf.global_variables_initializer() # 새변수 구변수 모두 초기화 어차피 자세한 변수들은 아래에서 불러오니깐 상관없음.
saver = tf.train.Saver()

In [367]:
cost_summary = tf.summary.scalar('cost', cost)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_batches = mnist_train_images_reduced.shape[0] // batch_size

In [369]:
with tf.Session() as sess:
    init.run()
    #### 이전 레이어 불러오기
    restore_saver.restore(sess,"./Ch11_HW_final.ckpt") # 여기에서 위에서 선언한 varibale들 다 불러옴.
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(mnist_train_images_reduced.shape[0])
        X_batches = np.array_split(mnist_train_images_reduced[shuffled_idx], n_batches)
        Y_batches = np.array_split(mnist_train_labels_reduced[shuffled_idx], n_batches)
        batch_index = 0
        for X_batch, Y_batch in zip(X_batches, Y_batches):
            sess.run(training_op, feed_dict={X:X_batch, Y:Y_batch})
            batch_index += 1
            loss = cost.eval(feed_dict={X:X_batch, Y:Y_batch}) # iteration 돌릴때 cost=cost.eval(~)f로 하면 중복되어버림 즉 cost가 numpy float으로 선언되어버림
            step = epoch * n_batches + batch_index
            summary_str = cost_summary.eval(feed_dict={X: X_batch, Y: Y_batch})
            file_writer.add_summary(summary_str, step)
        acc_train = accuracy.eval(feed_dict={X:X_batch, Y:Y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist_valid_images_reduced, Y:mnist_valid_labels_reduced})
        #print (epoch, "Train accuracy:", acc_train, "Cost:", loss)
        print(epoch, "Train accuracy:", acc_train, "Validation accuracy:", acc_val)
    save_path = saver.save(sess, "./Ch11_HW_final_new.ckpt")
    file_writer.close()

INFO:tensorflow:Restoring parameters from ./Ch11_HW_final.ckpt
0 Train accuracy: 0.984016 Validation accuracy: 0.9769351
1 Train accuracy: 0.991009 Validation accuracy: 0.9824081
2 Train accuracy: 0.992008 Validation accuracy: 0.9870993
3 Train accuracy: 0.997003 Validation accuracy: 0.9878812
4 Train accuracy: 1.0 Validation accuracy: 0.9882721
5 Train accuracy: 0.999001 Validation accuracy: 0.9878812
6 Train accuracy: 1.0 Validation accuracy: 0.98944485
7 Train accuracy: 1.0 Validation accuracy: 0.9898358
8 Train accuracy: 0.999001 Validation accuracy: 0.99022675
9 Train accuracy: 1.0 Validation accuracy: 0.9917905
10 Train accuracy: 1.0 Validation accuracy: 0.98944485
11 Train accuracy: 1.0 Validation accuracy: 0.98905396
12 Train accuracy: 1.0 Validation accuracy: 0.9910086
13 Train accuracy: 1.0 Validation accuracy: 0.99022675
14 Train accuracy: 1.0 Validation accuracy: 0.9906177
15 Train accuracy: 1.0 Validation accuracy: 0.99022675
16 Train accuracy: 1.0 Validation accuracy: 0.9

### 앞에서 meta파일을 세이버로 하면 안됨?
#### 안됨 그 이유는 import로 그래프를 불러오면 뒤에서 get_collection할 때 그래프가 중복되어 버림. 프린트해보면 _number가 붙는데 이말은 그래프가 중복해서 쌓였다는 이야기임

## 동결해보기 
`
train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|outputs")
training_op = optimizer.minizie(loss, var_list=train_vars)
`

다른방법도 가능    

`
hidden3_stop = tf.stop_gardient(hidden3)
`

In [394]:
tf.reset_default_graph()

In [395]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist=input_data.read_data_sets("MNIST")

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [396]:
reduced_train = mnist.train.labels < 5
reduced_valid = mnist.validation.labels < 5
#print (reduced_valid)
mnist_train_images_reduced = mnist.train.images[reduced_train]
mnist_train_labels_reduced = mnist.train.labels[reduced_train]
mnist_valid_images_reduced = mnist.validation.images[reduced_valid]
mnist_valid_labels_reduced = mnist.validation.labels[reduced_valid]

In [397]:
n_inputs = 28*28
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_hidden4 = 100
n_hidden5 = 100
n_outputs = 5

In [398]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name = "X") # 이미 scaled 되어있는 데이터가 들어갈 예정
Y = tf.placeholder(tf.int32, shape=(None), name="Y")

In [399]:
# DNN 만들어주는 파트, name_scope로 만들어주는 부분들의 이름들을 하나로 묶음

he_init = tf.contrib.layers.variance_scaling_initializer()
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, name="hidden3", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden3_stop = tf.stop_gradient(hidden2)
    hidden4 = tf.layers.dense(hidden3_stop, n_hidden4, name="hidden4", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, name="hidden5", activation=tf.nn.elu, kernel_initializer=he_init)
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs", kernel_initializer=he_init)
    logits_out = tf.nn.softmax(logits)

In [400]:
# DNN에서 평가를 위한 loss파트 정의
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
    cost = tf.reduce_mean(xentropy, name="cost")

In [401]:
# DNN 훈련파트
learning_rate = 0.01
with tf.name_scope("train"):
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    #train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|outputs")
    #print (train_vars)
    #training_op = optimizer.minimize(cost, var_list=train_vars)
    training_op = optimizer.minimize(cost)

In [402]:
## DNN 평가를 위한 파트
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, Y, 1) ## 
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) ## type을 bool에서 float32로 바꿔줌

In [403]:
n_epochs = 30
batch_size = 1000 ## 최대한 트레이닝 샘플 개수의 약수로 나오게 현재는 28038개

In [404]:
init = tf.global_variables_initializer() # 새변수 구변수 모두 초기화 어차피 자세한 변수들은 아래에서 불러오니깐 상관없음.
saver = tf.train.Saver()

In [405]:
cost_summary = tf.summary.scalar('cost', cost)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_batches = mnist_train_images_reduced.shape[0] // batch_size

In [406]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(mnist_train_images_reduced.shape[0])
        X_batches = np.array_split(mnist_train_images_reduced[shuffled_idx], n_batches)
        Y_batches = np.array_split(mnist_train_labels_reduced[shuffled_idx], n_batches)
        batch_index = 0
        for X_batch, Y_batch in zip(X_batches, Y_batches):
            sess.run(training_op, feed_dict={X:X_batch, Y:Y_batch})
            batch_index += 1
            loss = cost.eval(feed_dict={X:X_batch, Y:Y_batch}) # iteration 돌릴때 cost=cost.eval(~)f로 하면 중복되어버림 즉 cost가 numpy float으로 선언되어버림
            step = epoch * n_batches + batch_index
            summary_str = cost_summary.eval(feed_dict={X: X_batch, Y: Y_batch})
            file_writer.add_summary(summary_str, step)
        acc_train = accuracy.eval(feed_dict={X:X_batch, Y:Y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist_valid_images_reduced, Y:mnist_valid_labels_reduced})
        #print (epoch, "Train accuracy:", acc_train, "Cost:", loss)
        print(epoch, "Train accuracy:", acc_train, "Validation accuracy:", acc_val)
    save_path = saver.save(sess, "./Ch11_HW_final_new.ckpt")
    file_writer.close()

0 Train accuracy: 0.951049 Validation accuracy: 0.94448787
1 Train accuracy: 0.966034 Validation accuracy: 0.9601251
2 Train accuracy: 0.97402596 Validation accuracy: 0.9691165
3 Train accuracy: 0.98301697 Validation accuracy: 0.9769351
4 Train accuracy: 0.987013 Validation accuracy: 0.9784988
5 Train accuracy: 0.98301697 Validation accuracy: 0.9765442
6 Train accuracy: 0.994006 Validation accuracy: 0.9784988
7 Train accuracy: 0.986014 Validation accuracy: 0.9816263
8 Train accuracy: 0.988012 Validation accuracy: 0.9816263
9 Train accuracy: 0.99000996 Validation accuracy: 0.98358095
10 Train accuracy: 0.995005 Validation accuracy: 0.98279905
11 Train accuracy: 0.984016 Validation accuracy: 0.98319
12 Train accuracy: 0.995005 Validation accuracy: 0.98358095
13 Train accuracy: 0.997003 Validation accuracy: 0.98553556
14 Train accuracy: 0.998002 Validation accuracy: 0.98279905
15 Train accuracy: 0.996004 Validation accuracy: 0.98631746
16 Train accuracy: 1.0 Validation accuracy: 0.9859265

## 재사용하는 네트워크로 새로운 데이터 훈련해보기

In [540]:
tf.reset_default_graph()
#saver_meta = tf.train.import_meta_graph("./Ch11_HW_final.ckpt.meta")

In [541]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist=input_data.read_data_sets("MNIST")

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


새로운데이터 만들기

In [542]:
reduced_train_index = []
reduced_valid_index = []
num_data = 100
#print (type(mnist_train_images_reduced))
for i in range(5,10):
    for j in range(num_data):
        reduced_train_index.append(np.where((mnist.train.labels == int(i)) == True)[0][:num_data][j])
        reduced_valid_index.append(np.where((mnist.validation.labels == int(i)) == True)[0][:num_data][j])
mnist_train_images_reduced = mnist.train.images[reduced_train_index]
mnist_train_labels_reduced = mnist.train.labels[reduced_train_index] - 5
mnist_valid_images_reduced = mnist.validation.images[reduced_valid_index]
mnist_valid_labels_reduced = mnist.validation.labels[reduced_valid_index] -5

print (mnist_train_labels_reduced)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

In [543]:
n_inputs = 28*28
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_hidden4 = 100
n_hidden5 = 100
n_outputs = 5

In [544]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name = "X") # 이미 scaled 되어있는 데이터가 들어갈 예정
Y = tf.placeholder(tf.int32, shape=(None), name="Y")

In [545]:
# DNN 만들어주는 파트, name_scope로 만들어주는 부분들의 이름들을 하나로 묶음

he_init = tf.contrib.layers.variance_scaling_initializer()
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, name="hidden3", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden4 = tf.layers.dense(hidden3, n_hidden4, name="hidden4", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, name="hidden5", activation=tf.nn.elu, kernel_initializer=he_init)
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs", kernel_initializer=he_init)
    logits_out = tf.nn.softmax(logits)

In [546]:
# DNN에서 평가를 위한 loss파트 정의
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
    cost = tf.reduce_mean(xentropy, name="cost")

In [547]:
# DNN 훈련파트
learning_rate = 0.01
with tf.name_scope("train"):
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(cost)

In [548]:
## DNN 평가를 위한 파트
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, Y, 1) ## 
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) ## type을 bool에서 float32로 바꿔줌

In [555]:
n_epochs = 30
batch_size = 100 ## 최대한 트레이닝 샘플 개수의 약수로 나오게 현재는 28038개

In [556]:
### 재사용할 레이어 불러오기
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="hidden[12345]") # scope에 아무것도 안하면 전체 다 불러옴 즉 마지막 출력층 (ouput)까지 불러옴
print (reuse_vars) # 출력해보면 어떤게 나오는지 감이 잡힘
restore_saver = tf.train.Saver(reuse_vars)

[<tf.Variable 'hidden1/kernel:0' shape=(784, 100) dtype=float32_ref>, <tf.Variable 'hidden1/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden2/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'hidden2/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden3/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'hidden3/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden4/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'hidden4/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden5/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'hidden5/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden1/kernel/Adam:0' shape=(784, 100) dtype=float32_ref>, <tf.Variable 'hidden1/kernel/Adam_1:0' shape=(784, 100) dtype=float32_ref>, <tf.Variable 'hidden1/bias/Adam:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden1/bias/Adam_1:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'hidden2/kernel/Adam:0' shape=(100, 10

In [557]:
init = tf.global_variables_initializer() # 새변수 구변수 모두 초기화 어차피 자세한 변수들은 아래에서 불러오니깐 상관없음.
saver = tf.train.Saver()

In [558]:
cost_summary = tf.summary.scalar('cost', cost)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_batches = mnist_train_images_reduced.shape[0] // batch_size
print (n_batches)

5


In [559]:
with tf.Session() as sess:
    init.run()
    #### 이전 레이어 불러오기
    restore_saver.restore(sess,"./Ch11_HW_final.ckpt") # 여기에서 위에서 선언한 varibale들 다 불러옴.
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(mnist_train_images_reduced.shape[0])
        X_batches = np.array_split(mnist_train_images_reduced[shuffled_idx], n_batches)
        Y_batches = np.array_split(mnist_train_labels_reduced[shuffled_idx], n_batches)
        batch_index = 0
        for X_batch, Y_batch in zip(X_batches, Y_batches):
            sess.run(training_op, feed_dict={X:X_batch, Y:Y_batch})
            batch_index += 1
            loss = cost.eval(feed_dict={X:X_batch, Y:Y_batch}) # iteration 돌릴때 cost=cost.eval(~)f로 하면 중복되어버림 즉 cost가 numpy float으로 선언되어버림
            step = epoch * n_batches + batch_index
            summary_str = cost_summary.eval(feed_dict={X: X_batch, Y: Y_batch})
            file_writer.add_summary(summary_str, step)
        acc_train = accuracy.eval(feed_dict={X:X_batch, Y:Y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist_valid_images_reduced, Y:mnist_valid_labels_reduced})
        #print (epoch, "Train accuracy:", acc_train, "Cost:", loss)
        print(epoch, "Train accuracy:", acc_train, "Validation accuracy:", acc_val)
    save_path = saver.save(sess, "./Ch11_HW_final_new.ckpt")
    file_writer.close()

INFO:tensorflow:Restoring parameters from ./Ch11_HW_final.ckpt
0 Train accuracy: 0.64 Validation accuracy: 0.62
1 Train accuracy: 0.89 Validation accuracy: 0.83
2 Train accuracy: 0.88 Validation accuracy: 0.804
3 Train accuracy: 0.91 Validation accuracy: 0.862
4 Train accuracy: 0.96 Validation accuracy: 0.888
5 Train accuracy: 0.95 Validation accuracy: 0.864
6 Train accuracy: 0.99 Validation accuracy: 0.904
7 Train accuracy: 0.96 Validation accuracy: 0.878
8 Train accuracy: 1.0 Validation accuracy: 0.91
9 Train accuracy: 0.98 Validation accuracy: 0.92
10 Train accuracy: 0.98 Validation accuracy: 0.902
11 Train accuracy: 0.99 Validation accuracy: 0.916
12 Train accuracy: 1.0 Validation accuracy: 0.93
13 Train accuracy: 1.0 Validation accuracy: 0.928
14 Train accuracy: 1.0 Validation accuracy: 0.928
15 Train accuracy: 1.0 Validation accuracy: 0.924
16 Train accuracy: 1.0 Validation accuracy: 0.928
17 Train accuracy: 1.0 Validation accuracy: 0.926
18 Train accuracy: 1.0 Validation accurac

In [560]:
with tf.Session() as sess:
    init.run()
    #### 이전 레이어 불러오기
    #restore_saver.restore(sess,"./Ch11_HW_final.ckpt") # 여기에서 위에서 선언한 varibale들 다 불러옴.
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(mnist_train_images_reduced.shape[0])
        X_batches = np.array_split(mnist_train_images_reduced[shuffled_idx], n_batches)
        Y_batches = np.array_split(mnist_train_labels_reduced[shuffled_idx], n_batches)
        batch_index = 0
        for X_batch, Y_batch in zip(X_batches, Y_batches):
            sess.run(training_op, feed_dict={X:X_batch, Y:Y_batch})
            batch_index += 1
            loss = cost.eval(feed_dict={X:X_batch, Y:Y_batch}) # iteration 돌릴때 cost=cost.eval(~)f로 하면 중복되어버림 즉 cost가 numpy float으로 선언되어버림
            step = epoch * n_batches + batch_index
            summary_str = cost_summary.eval(feed_dict={X: X_batch, Y: Y_batch})
            file_writer.add_summary(summary_str, step)
        acc_train = accuracy.eval(feed_dict={X:X_batch, Y:Y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist_valid_images_reduced, Y:mnist_valid_labels_reduced})
        #print (epoch, "Train accuracy:", acc_train, "Cost:", loss)
        print(epoch, "Train accuracy:", acc_train, "Validation accuracy:", acc_val)
    save_path = saver.save(sess, "./Ch11_HW_final_new.ckpt")
    file_writer.close()

0 Train accuracy: 0.42 Validation accuracy: 0.438
1 Train accuracy: 0.83 Validation accuracy: 0.752
2 Train accuracy: 0.89 Validation accuracy: 0.824
3 Train accuracy: 0.93 Validation accuracy: 0.838
4 Train accuracy: 0.94 Validation accuracy: 0.872
5 Train accuracy: 0.98 Validation accuracy: 0.866
6 Train accuracy: 1.0 Validation accuracy: 0.892
7 Train accuracy: 0.97 Validation accuracy: 0.882
8 Train accuracy: 0.98 Validation accuracy: 0.908
9 Train accuracy: 1.0 Validation accuracy: 0.866
10 Train accuracy: 1.0 Validation accuracy: 0.912
11 Train accuracy: 0.97 Validation accuracy: 0.856
12 Train accuracy: 1.0 Validation accuracy: 0.9
13 Train accuracy: 1.0 Validation accuracy: 0.894
14 Train accuracy: 1.0 Validation accuracy: 0.9
15 Train accuracy: 1.0 Validation accuracy: 0.902
16 Train accuracy: 1.0 Validation accuracy: 0.906
17 Train accuracy: 1.0 Validation accuracy: 0.898
18 Train accuracy: 1.0 Validation accuracy: 0.902
19 Train accuracy: 1.0 Validation accuracy: 0.904
20 Tr